In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import plotly.express as px
from tqdm import tqdm
import random

import optuna
from optuna.samplers import RandomSampler
from optuna.samplers import TPESampler

optuna.logging.set_verbosity(optuna.logging.ERROR)

In [5]:
import sys
sys.path.append('../src')

import vrpp
import importlib
importlib.reload(vrpp)

<module 'vrpp' from '/Users/sergak/PycharmProjects/optimal_encashement/notebooks/../src/vrpp.py'>

In [3]:
dist = pd.read_csv('../data/raw/times v4.csv')
le = preprocessing.LabelEncoder()
le.fit(dist['Origin_tid'])
dist['from_int'] = le.transform(dist['Origin_tid'])
dist['to_int'] = le.transform(dist['Destination_tid'])
dist.head()

,Origin_tid,Destination_tid,Total_Time,from_int,to_int
0,636538,683103,15.32,864,1354
1,636538,634763,16.20,864,624
2,636538,683128,16.27,864,1358
3,636538,683789,16.77,864,1374
4,636538,634709,17.67,864,603


In [4]:
data = pd.read_excel('../data/raw/terminal_data_hackathon v4.xlsx', 'Incomes')
data.head()

,TID,остаток на 31.08.2022 (входящий),2022-09-01 00:00:00,2022-09-02 00:00:00,2022-09-03 00:00:00,2022-09-04 00:00:00,2022-09-05 00:00:00,2022-09-06 00:00:00,2022-09-07 00:00:00,2022-09-08 00:00:00,...,2022-11-21 00:00:00,2022-11-22 00:00:00,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00,2022-11-28 00:00:00,2022-11-29 00:00:00,2022-11-30 00:00:00
0,406136,160000,90000,105000,99000,107000,110000,60000,75000,89000,...,91000,78000,0,165000,0,189000,106000,94000,75000,74000
1,406139,387000,103000,206000,168000,124000,78000,165000,164000,174000,...,164000,153000,151000,157000,206000,182000,123000,138000,112000,179000
2,406145,287000,143000,136000,124000,117000,123000,140000,139000,138000,...,119000,100000,179000,169000,118000,118000,114000,128000,121000,124000
3,406148,355000,50000,73000,53000,65000,75000,100000,53000,52000,...,48000,55000,65000,85000,95000,68000,62000,0,118000,70000
4,406180,597000,96000,82000,71000,72000,86000,55000,55000,75000,...,82000,56000,70000,59000,105000,70000,77000,87000,59000,55000


In [5]:
config = {'num_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': 100,
          'terminal_service_persent': 0, #0.01
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10,
          'left_days_coef': 0,
          'encashment_coef': 0,}

In [12]:
real_cash = (data[data.columns[1:]].values * (1 + 0.2 * (random.random() - 0.5))).copy()
# real_cash = data[data.columns[1:]].values.copy()
cash = real_cash.copy()

In [13]:
cash = cash.astype(int)
cash.shape

(1630, 92)

In [14]:
INF = 1e9

In [ ]:
def get_cost(days_left, delta_loss):
    if days_left == 0:
        return INF
    soon_deadline_loss = 2 ** (config['max_not_service_days'] - days_left)
    return soon_deadline_loss + delta_loss * config['delta_loss_coef']

In [17]:
def get_cost(days_left):
    if days_left == 0: return INF
    return 2 ** (config['max_not_service_days'] - days_left)

In [16]:
cash = real_cash.copy()
num_vehicles = 5
num_terminals = cash.shape[0]
time_until_force = [config['max_not_service_days'] for i in range(num_terminals)]
cur_cash = cash[:, 0]
cash = cash[:, 1:]

hist = {'visited': [], 'loss': []}
bad = False

myvrp = vrpp.VRPP(dist, 10, 10 * 60, num_vehicles, solution_limit=100, time_limit=100, dead_loss=False)

days = 90
big_id = config['max_not_service_days']
for day in tqdm(range(days)):
    mask = []
    cost = []
    to_counter = [0 for i in range(config['max_not_service_days'] + 1)]
    
    for i in range(num_terminals):
        force = time_until_force[i]
        if cur_cash[i] > config['max_terminal_money']:
            force = 0
            
            
        force_for_show = time_until_force[i]
        current_money = cur_cash[i]
        for forecast in range(min(force_for_show, days - day)):
            if current_money > config['max_terminal_money']:
                force_for_show = forecast
                break
            if not forecast:
                current_money += cash[i][day + forecast]
            else:
                current_money += cash[i][day + forecast] # check later
                
        mask.append(1)
        cost.append(int(get_cost(force)))
        to_counter[force_for_show] += 1
    
    print(to_counter)
    # # for i in range(100):
    # #     print(" " * (4 - len(str(cost[i]))) + str(cost[i]), end=' ')
    # # print()
    # print(" "cost[:100])
    
    visited, paths = myvrp.find_vrp(cost, mask)
    hist['visited'].append(visited)
    
    prev = []
    cur_loss = 0
    for i in range(num_terminals):
        if cur_cash[i] > config['max_terminal_money'] or time_until_force[i] == 0:
            if not visited[i]:
                print('Dead')
                bad = True
                break

        if visited[i]:
            cur_loss += max(config['terminal_service_cost'], cur_cash[i] * config['terminal_service_persent'])
            cur_cash[i] = 0
            time_until_force[i] = config['max_not_service_days']
        else:
            time_until_force[i] -= 1
            
        cur_loss += cur_cash[i] * config['persent_day_income']
        cur_cash[i] += cash[i][day]
    
    if bad:
        break
    
    hist['loss'].append(cur_loss)
    big_id -= 1
    
print(sum(hist['loss']))

  0%|                                                                                                          | 0/90 [00:00<?, ?it/s]

[10, 7, 22, 37, 35, 43, 63, 62, 63, 79, 95, 75, 86, 73, 880]


  1%|█                                                                                                 | 1/90 [00:19<29:25, 19.84s/it]

[6, 18, 30, 34, 38, 63, 60, 62, 74, 87, 70, 84, 77, 752, 175]


  2%|██▏                                                                                               | 2/90 [00:37<27:17, 18.61s/it]

...

[22, 28, 0, 38, 105, 129, 134, 132, 140, 146, 152, 165, 143, 151, 145]


 99%|███████████████████████████████████████████████████████████████████████████████████████████████▉ | 89/90 [25:36<00:18, 18.35s/it]

[22, 0, 0, 49, 122, 133, 132, 140, 146, 152, 165, 143, 151, 145, 130]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [26:12<00:00, 17.47s/it]

3472341.908643171


## Look json

In [23]:
!ls

VRPwP-Copy1.ipynb        catboost_info            tid_mean.pkl
VRPwP.ipynb              demand_prediction.ipynb  vrp.ipynb
about_final-sergey.ipynb income_experiments.ipynb zero_aggregation.pkl
about_final.ipynb        optuna.ipynb


In [11]:
import json

In [12]:
with open('../data/processed/raw_report_4_dp.json', 'r') as f:
    data = json.load(f)
    for i in range(len(data['logs'])):
        for k, v in data['logs'][i].items():
            if type(v) == list:
                data['logs'][i][k] = np.array(v)

/var/folders/cz/4mm9fww57k9fxrmn10mvt2_h0000gn/T/ipykernel_13036/322514209.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data['logs'][i][k] = np.array(v)


In [13]:
sum([el['losses'] for el in data['logs']])

10910242.376712326

In [19]:
(data['logs'][0]['dp'] == 50).sum()

1630

In [26]:
sum([el['losses'] for el in data['logs'][:30]])

3597489.794520548

In [34]:
sum([el['losses'] for el in data['logs'][:50]]) - 5994289

-172.14794520661235

In [30]:
import collections

collections.Counter(data['logs'][12]['days_until_death']).most_common()

[(4, 181),
 (6, 160),
 (5, 157),
 (7, 153),
 (8, 150),
 (14, 131),
 (9, 130),
 (10, 124),
 (11, 115),
 (12, 92),
 (13, 79),
 (3, 63),
 (1, 49),
 (2, 43),
 (0, 3)]

In [36]:
for i in range(len(data['logs'])):
    if data['logs'][i]['loss'] > 1e9:
        idx = i
        break

In [37]:
data['logs'][0].keys()

dict_keys(['loss', 'visited', 'paths', 'costs', 'dayes_until_death'])

In [38]:
import numpy as np
np.unique(data['logs'][0]['dayes_until_death'])

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [39]:
sum(data['logs'][idx]['visited'])

123

In [40]:
idx

64

In [50]:
data['logs'][idx]['dayes_until_death'].argmin()

107446

In [48]:
mask = data['logs'][idx]['dayes_until_death'] != -1
np.bincount(data['logs'][idx]['dayes_until_death'][mask], wei)

array([  707,  7800, 10220, 10962, 11385, 11587, 11519, 11355, 11116,
       10972, 10714, 10411, 10112,  9840,  9628])

In [49]:
data['logs'][idx]['dayes_until_death'].shape

(148330,)

In [45]:
data['logs'][idx]['dayes_until_death'].min()

-1

In [46]:
for i in range(-3):
    print(i)

In [33]:
data['logs'][idx - 1]

{'loss': 119214.30273972607,
 'visited': [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [12]:
sum(data['logs'][idx]['visited'])

123

In [13]:
sum(data['logs'][idx + 1]['visited'])

132